<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h2>Relocation Decision Assistance</h2>

<h3>1. Introduction</h3>

<h4>1.1. Background</h4>

According to the US Census Bureau about 40 million people (or 14% of US population) move every year. People are moving for different reasons: job, family, school, cost of living, etc. But despite the reason the questions people face when they are looking for a new place to live are very common. Is the housing price in the new location affordable? Will I have same or better amenities in my new neighborhood? What is the quality of public schools? The purpose of this project is to help answer those questions with the use of the data science.

<h4>1.2. Problem</h4>

These days, Internet is great source of information on the topic of interest. But despite wide resources or sometimes because of the resource abundance, the answers may not be so easy to find. For example, a person would need to spend hours researching a single neighborhood and may not even investigate other close by areas that might be more suitable for relocation. In some cases, a person may not even know where to start looking. This project is aimed to guide the relocation decision making by showing clusters of suitable neighborhoods according to provided criteria. 

To demonstrate the results, I will study a specific case. A hypothetical family is happy with their current location but want to be closer to their family. Therefore, they want to relocate to the state of MA. Their criteria are good public schools, affordable housing and good amenities – just like their current place.  

<h4>1.3. Interest</h4>

The results of the project will be equally interesting for real estate agents as well as their clients. 

<h3>2. Data acquisition and cleaning</h3>

<h4>2.1. Data sources</h4>

These project uses three data sources:<ul>
<li> USZipCode database available on <a href="https://pypi.org/project/uszipcode/">PyPI</a>. The database provides geographical, statistical, real-estate data for every US zip code.</li>
<li> <a href="https://foursquare.com/">FourSquare</a> data for information on amenities. FourSquare API is used to collect the data.</li>
<li> <a href="https://www.greatschools.org/">GreatSchools</a> web site for public schools rating. I wrote a function to scrape the web site and to get the average schools rating for the provided zip code.</li>

<b>USZipCode database exploration</b>

In [39]:
# Install PyPI library
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\olgazaychykova\anaconda3\lib\site-packages (19.2.1)


In [40]:
# Install USzipcode database
# !pip install uszipcode # uncomment this line if the notebook is ran for the first time.

Explore data available for the target zip code 94598. This is the zip code of the hypothetical family.

In [41]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database
targetZip=search.by_zipcode("94598")
targetZip=targetZip.to_dict() # convert to dictionary
targetZip # check the info

{'area_code_list': ['510', '707', '925'],
 'bounds_east': -121.925567,
 'bounds_north': 37.942412,
 'bounds_south': 37.849998,
 'bounds_west': -122.05489,
 'common_city_list': ['Walnut Creek'],
 'county': 'Contra Costa County',
 'housing_units': 10756,
 'land_area_in_sqmi': 15.25,
 'lat': 37.91,
 'lng': -122.05,
 'major_city': 'Walnut Creek',
 'median_home_value': 719200,
 'median_household_income': 121067,
 'occupied_housing_units': 10390,
 'population': 25818,
 'population_density': 1693.0,
 'post_office_city': 'Walnut Creek, CA',
 'radius_in_miles': 6.0,
 'state': 'CA',
 'timezone': 'Pacific',
 'water_area_in_sqmi': 0.01,
 'zipcode': '94598',
 'zipcode_type': 'Standard'}

The following geo information will be used for scraping GreatSchools web site: post office city, latitude, longitude, state, radius, zip code. But first, lets visualize this location on the map.

In [42]:
# Install and import map libraries
# !pip install folium # uncomment this line if the notebook is ran first time
import folium # map rendering library

In [43]:
# create map of using latitude and longitude values of the target zip code
lat=targetZip["lat"]
lng=targetZip["lng"]
map_targetZip = folium.Map(location=[lat, lng], zoom_start=5)

# add marker to the map
label = '{}, {}'.format(targetZip["post_office_city"], targetZip["zipcode"])
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
       [lat, lng],
       radius=5,
       popup=label,
       color='blue',
       fill=True,
       fill_color='#3186cc',
       fill_opacity=0.7,
       parse_html=False).add_to(map_targetZip)  
    
map_targetZip

<b>Scrape GreatSchools site</b>

In [44]:
# Import libraries
import pandas as pd
import requests
import math
from bs4 import BeautifulSoup

# Create function to scrape GreatSchools
# The function return average rating of all selected grade (elementary, middle, high) public schools 
# within 5 miles radius of specified zipcode.
# Variables description:
#      lat = latitude
#      lon = longitude
#      rad = radius in miles
#      locationLabel = City%20Name%2C%20State (%20 is the space and %2C%20 is comma space). Example: San%20Francisco%2C%20CA
#      gradeLevels = one of the following: e, m, h
#      state = two letter state code
#      zipCode = 5 digits zip code
def scrape_GreatSchools(lat,lon,rad,locationLabel,gradeLevels,state,zipCode):
    gsURL='https://www.greatschools.org/search/search.page?distance='+rad+'&lat='+lat+'&lon='+lon+'&locationLabel='+locationLabel+'%20'+zipCode+'&locationType=zip&st=public_charter&st=public&gradeLevels='+gradeLevels+'&state='+state+'&sort=distance&view=list'
    res = requests.get(gsURL)
    soup = BeautifulSoup(res.content, 'html.parser')
    # Convert soup object to string and get only the substring with school ratings
    soup_string=str(soup)
    soup_string.find('gon.search={"schools":[')
    soup_string.find('"resultSummary":')
    schools=soup_string[soup_string.find('gon.search={"schools":[')+len('gon.search={"schools":['):soup_string.find('"resultSummary":')]
    # Split by '"remediationData":[]}' string
    schools_split=schools.split('"remediationData":[]}')
    # build cleaned up arrays for zip and rating
    i = 0
    rating=[] # empty rating array
    while i < len(schools_split)-1:
        if schools_split[i][schools_split[i].find('"rating":')+len('"rating":'):schools_split[i].find(',"ratingScale"')]=='null':
            i+=1
        else:
            rating.append(schools_split[i][schools_split[i].find('"rating":')+len('"rating":'):schools_split[i].find(',"ratingScale"')])
            i += 1
    # find average rating for schools
    i=0 
    sum=0
    n=0
    while i<len(rating):
        sum+=int(rating[i])
        n+=1
        i+=1
    if n==0:
        return -1
    else:
        return math.floor(sum/n)

Scrape GreatSchools with target zip code.

In [45]:
#Test function with targetZip
lat=str(targetZip["lat"])
lng=str(targetZip["lng"])
rad='5' #look for schools within 5 miles radius of the zip code
city=targetZip["post_office_city"].replace(' ','%20').replace(',','%2C')
state=targetZip["state"]
zipcode=str(targetZip["zipcode"])
print('Average elementary schools rating: ',scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode))
print('Average middle schools rating: ',scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode))
print('Average high schools rating: ',scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode))

Average elementary schools rating:  6
Average middle schools rating:  7
Average high schools rating:  7


In [46]:
# add elementary schools rating info to the dictionary
targetZip["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
#targetZip["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
#targetZip["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
targetZip

{'area_code_list': ['510', '707', '925'],
 'bounds_east': -121.925567,
 'bounds_north': 37.942412,
 'bounds_south': 37.849998,
 'bounds_west': -122.05489,
 'common_city_list': ['Walnut Creek'],
 'county': 'Contra Costa County',
 'e_school': 6,
 'housing_units': 10756,
 'land_area_in_sqmi': 15.25,
 'lat': 37.91,
 'lng': -122.05,
 'major_city': 'Walnut Creek',
 'median_home_value': 719200,
 'median_household_income': 121067,
 'occupied_housing_units': 10390,
 'population': 25818,
 'population_density': 1693.0,
 'post_office_city': 'Walnut Creek, CA',
 'radius_in_miles': 6.0,
 'state': 'CA',
 'timezone': 'Pacific',
 'water_area_in_sqmi': 0.01,
 'zipcode': '94598',
 'zipcode_type': 'Standard'}

<b>Explore FourSquare API with target zip code</b>

In [47]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'QGKUC2UWM2XORZTDMPZLZCQP55HC325UHK25KFJQOKP5LHWH' # your Foursquare ID
CLIENT_SECRET = 'PMXKF3LGTXVJEGBVKH2XAUUJ23K5JRLKENWF053MRJXI0ISB' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

In [48]:
#Create GET request URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = int(targetZip["radius_in_miles"])*1609 # define radius in meters
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QGKUC2UWM2XORZTDMPZLZCQP55HC325UHK25KFJQOKP5LHWH&client_secret=PMXKF3LGTXVJEGBVKH2XAUUJ23K5JRLKENWF053MRJXI0ISB&v=20180604&ll=37.91,-122.05&radius=9654&limit=100'

In [49]:
#Send the GET request and examine the resutls.
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d481f109929510025cdbf93'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4a084f23f964a520ae731fe3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4a084f23f964a520ae731fe3',
       'location': {'address': '301 N San Carlos Dr',
        'cc': 'US',
        'city': 'Walnut Creek',
        'country': 'United States',
        'crossStreet': 'at Ygnacio Valley Rd',
        'distance': 1196,
        'formattedAddress': ['301 N San Carlos Dr (at Ygnacio Valley Rd)',
         'Walnut Creek, CA 94598',
         'United St

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Heather Farm Park,Park,37.918570,-122.041768
1,Gardens at Heather Farm,Garden,37.918820,-122.044038
2,Lottie's Creamery,Ice Cream Shop,37.899577,-122.060806
3,Sports Basement,Sporting Goods Shop,37.918327,-122.036975
4,Montecatini Ristorante,Italian Restaurant,37.901636,-122.062617


In [52]:
# Print number of venues returned by Foursquare.
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<h4>2.2. Data cleaning</h4>

The data for the desired relocation area (state of MA) is collected from the USZipCode data. I don't need locations that are lacking information about median house price. Therefore, those locations are excluded.

In [53]:
# Get all zip codes for the state of MA
relocateInfo = search.by_state('MA',returns=500)

# Convert output to the list of dictionaries
relocateZip=[]
for y in relocateInfo:
    relocateZip.append(y.to_dict())
len(relocateZip)

496

In [56]:
# remove locations with missing median home value
i=0
while i<len(relocateZip):
    if relocateZip[i]["median_home_value"]==None: # exclude location with missing median home value        
        del relocateZip[i]
    i+=1
len(relocateZip)

481

Visualize locations on the map.

In [57]:
# Create a map of MA with zip codes superimposed on top.
map_MA = folium.Map(location=[42.292, -71.5], zoom_start=8)

# add markers to map
i=0
while i<len(relocateZip): 
    label = '{}, {}'.format(relocateZip[i]["major_city"], relocateZip[i]["zipcode"])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [relocateZip[i]["lat"], relocateZip[i]["lng"]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_MA)  
    i+=1
    
map_MA

To narrow down the potential relocation areas, I will remove zip codes that have median house price above the desired target. For the target price, I will take house median price in the current location of the client. 

In [60]:
# Remove locations with median house price above the price that client can afford.
i=0
while i<len(relocateZip):
    if relocateZip[i]["median_home_value"]>targetZip["median_home_value"]: 
        del relocateZip[i]
    i+=1
len(relocateZip)


463

Scrape GreatSchools for each of the zip codes to get the average schools rating. Add schools rating to each dictionary in the list. I have to limit the scraping function call to 50-100 consecutive cals, otherwise I get disconnected from the web site.

In [62]:
i=0
while i<50:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [63]:
i=50
while i<100:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [64]:
i=100
while i<150:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [65]:
i=150
while i<200:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [66]:
i=200
while i<250:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [67]:
i=250
while i<300:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [68]:
i=300
while i<350:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [69]:
i=350
while i<400:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [70]:
i=400
while i<450:
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


In [71]:
i=450
while i<len(relocateZip):
    lat=str(relocateZip[i]["lat"])
    lng=str(relocateZip[i]["lng"])
    rad='5'
    city=relocateZip[i]["post_office_city"].replace(' ','%20').replace(',','%2C')
    state=relocateZip[i]["state"]
    zipcode=str(relocateZip[i]["zipcode"])
    relocateZip[i]["e_school"]=scrape_GreatSchools(lat,lng,rad,city,'e',state,zipcode)
    #relocateZip[i]["m_school"]=scrape_GreatSchools(lat,lng,rad,city,'m',state,zipcode)
    #relocateZip[i]["h_school"]=scrape_GreatSchools(lat,lng,rad,city,'h',state,zipcode)
    i+=1
print('done')

done


Next, I will add target zip to the list and convert final result into dataframe.

In [72]:
# add targetZip to the list since it also will be used for future clustering.
relocateZip.append(targetZip)
len(relocateZip)

464

In [73]:
# Convert list to the dataframe
df = pd.DataFrame(relocateZip)
df.head()

,area_code_list,bounds_east,bounds_north,bounds_south,bounds_west,common_city_list,county,e_school,housing_units,land_area_in_sqmi,...,occupied_housing_units,population,population_density,post_office_city,radius_in_miles,state,timezone,water_area_in_sqmi,zipcode,zipcode_type
0,[413],-72.582535,42.100467,42.030795,-72.667902,[Agawam],Hampden County,4,7557,11.44,...,7215,16769,1466.0,"Agawam, MA",3.0,MA,Eastern,0.86,01001,Standard
1,[413],-72.355041,42.437947,42.301437,-72.546776,"[Amherst, Cushman, Pelham]",Hampshire County,4,10388,55.04,...,9910,29049,528.0,"Amherst, MA",7.0,MA,Eastern,1.65,01002,Standard
2,[978],-72.007388,42.484473,42.356423,-72.205174,[Barre],Worcester County,4,2044,44.24,...,1904,5079,115.0,"Barre, MA",7.0,MA,Eastern,0.26,01005,Standard
3,[413],-72.331642,42.358762,42.185812,-72.472287,[Belchertown],Hampshire County,3,5839,52.64,...,5595,14649,278.0,"Belchertown, MA",8.0,MA,Eastern,2.68,01007,Standard
4,[413],-72.872180,42.251340,42.113028,-73.034916,[Blandford],Hampden County,-1,586,53.80,...,503,1263,23.0,"Blandford, MA",7.0,MA,Eastern,1.96,01008,Standard


<h4>2.3. Feature Selection</h4>

Current data set has 25 features but I only need few in order to obtain data from FourSquare API and for future clustering. Thus, I will keep the following features that will be used for clustering: e_school,and median_home_value. The following features are needed for FourSquare API and record identification: zipcode lat, lng, post_office_city.

In [74]:
# create new dataframe with required features
dfFeature=df[['zipcode','post_office_city','lat','lng','e_school','median_home_value']]
dfFeature.head()

,zipcode,post_office_city,lat,lng,e_school,median_home_value
0,01001,"Agawam, MA",42.07,-72.63,4,213000
1,01002,"Amherst, MA",42.38,-72.52,4,338900
2,01005,"Barre, MA",42.42,-72.12,4,208500
3,01007,"Belchertown, MA",42.30,-72.40,3,260000
4,01008,"Blandford, MA",42.20,-73.00,-1,247200


In [75]:
# get some basic statistics about the dataframe
dfFeature.describe(include='all')

,zipcode,post_office_city,lat,lng,e_school,median_home_value
count,464,464,464.000000,464.000000,464.000000,464.000000
unique,464,403,NaN,NaN,NaN,NaN
top,01852,"Boston, MA",NaN,NaN,NaN,NaN
freq,1,10,NaN,NaN,NaN,NaN
mean,NaN,NaN,42.243129,-71.661545,4.308190,338721.120690
std,NaN,NaN,0.366382,2.474838,1.686517,127844.459974
min,NaN,NaN,37.910000,-122.050000,-1.000000,64000.000000
25%,NaN,NaN,42.090000,-72.112500,4.000000,247175.000000
50%,NaN,NaN,42.290000,-71.295000,5.000000,313300.000000
75%,NaN,NaN,42.470000,-71.017500,5.000000,398650.000000


In [76]:
# drop those zipcodes where school rating below target
dfFeatureClean=dfFeature[dfFeature["e_school"]>=targetZip["e_school"]]

In [77]:
dfFeatureClean.head()

,zipcode,post_office_city,lat,lng,e_school,median_home_value
5,01010,"Brimfield, MA",42.15,-72.20,6,264800
26,01054,"Leverett, MA",42.47,-72.49,6,341700
40,01081,"Wales, MA",42.06,-72.23,7,204300
72,01238,"Lee, MA",42.30,-73.22,6,251300
77,01254,"Richmond, MA",42.38,-73.37,6,404800


Get FourSquare data and construct a dataframe of venues.

In [78]:
# Define function to get venues within about 5 miles (8000 meters) radius of the zipcodes.
def getNearbyVenues(names, latitudes, longitudes, radius=8000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
#Run the above function on each zipcode and create a new dataframe called location_venues.
location_venues = getNearbyVenues(names=dfFeatureClean['zipcode'],
                                   latitudes=dfFeatureClean['lat'],
                                   longitudes=dfFeatureClean['lng']
                                  )

01010
01054
01081
01238
01254
01338
01341
01360
01370
01431
01432
01434
01451
01460
01463
01506
01515
01516
01521
01522
01532
01536
01541
01568
01581
01612
01718
01719
01720
01740
01742
01746
01747
01748
01752
01754
01757
01775
01776
01778
01801
01864
01867
01880
01886
01890
01929
01940
01949
01982
01983
02019
02032
02038
02043
02045
02050
02052
02053
02054
02056
02061
02062
02066
02071
02081
02093
02169
02188
02189
02191
02324
02332
02360
02420
02421
02461
02462
02464
02465
02466
02474
02476
02492
02494
02536
02540
02556
02557
02568
02631
02642
02649
02653
02657
02659
02703
02718
02739
02769
02770
02771
94598


In [80]:
# Check new dataframe
print(location_venues.shape)
location_venues.head()

(8774, 7)


,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,01010,42.15,-72.2,Brimfield Antique Show,42.121464,-72.209101,Antique Shop
1,01010,42.15,-72.2,Brimfield Antiques Center,42.120812,-72.213299,Antique Shop
2,01010,42.15,-72.2,Rapscallion Brewery,42.139765,-72.109926,Brewery
3,01010,42.15,-72.2,Old Village Grille,42.116750,-72.117853,Breakfast Spot
4,01010,42.15,-72.2,Simple Indulgence Day Spa,42.116110,-72.115648,Spa


In [82]:
# Number of venues were returned for each Zipcode
location_venues.groupby('Zipcode').count()

,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zipcode,,,,,,
01010,30,30,30,30,30,30
01054,24,24,24,24,24,24
01081,21,21,21,21,21,21
01238,83,83,83,83,83,83
01254,82,82,82,82,82,82
01338,20,20,20,20,20,20
01341,19,19,19,19,19,19
01360,12,12,12,12,12,12
01370,25,25,25,25,25,25


In [83]:
#How many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 321 uniques categories.


Analyze venues in each zip code.

In [84]:
# one hot encoding
location_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add zipcode column back to dataframe
location_onehot['Zipcode'] = location_venues['Zipcode'] 

# move neighborhood column to the first column
fixed_columns = [location_onehot.columns[-1]] + list(location_onehot.columns[:-1])
location_onehot = location_onehot[fixed_columns]

location_onehot.head()

,Zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Aquarium,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
0,01010,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01010,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [86]:
location_grouped = location_onehot.groupby('Zipcode').mean().reset_index()
location_grouped.head()

,Zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Aquarium,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
0,01010,0.0,0.000000,0.0,0.0,0.000000,0.0,0.066667,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.033333,0.0
1,01054,0.0,0.000000,0.0,0.0,0.041667,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.041667,0.0
2,01081,0.0,0.000000,0.0,0.0,0.000000,0.0,0.095238,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,01238,0.0,0.036145,0.0,0.0,0.060241,0.0,0.012048,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.012048,0.000000,0.0
4,01254,0.0,0.000000,0.0,0.0,0.024390,0.0,0.012195,0.0,0.0,...,0.012195,0.0,0.0,0.0,0.0,0.012195,0.0,0.000000,0.012195,0.0


Add school rating, median home price, and venues into one dataframe for future clustering.

In [89]:
location_grouped.rename(columns={'Zipcode': 'zipcode'}, inplace=True)
location_grouped.head()
print(location_grouped.shape)

(103, 322)


In [90]:
dfCluster=pd.merge(location_grouped, dfFeatureClean, on='zipcode')
dfCluster.head()

,zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Aquarium,...,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,post_office_city,lat,lng,e_school,median_home_value
0,01010,0.0,0.000000,0.0,0.0,0.000000,0.0,0.066667,0.0,0.0,...,0.000000,0.0,0.000000,0.033333,0.0,"Brimfield, MA",42.15,-72.20,6,264800
1,01054,0.0,0.000000,0.0,0.0,0.041667,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.041667,0.0,"Leverett, MA",42.47,-72.49,6,341700
2,01081,0.0,0.000000,0.0,0.0,0.000000,0.0,0.095238,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,"Wales, MA",42.06,-72.23,7,204300
3,01238,0.0,0.036145,0.0,0.0,0.060241,0.0,0.012048,0.0,0.0,...,0.000000,0.0,0.012048,0.000000,0.0,"Lee, MA",42.30,-73.22,6,251300
4,01254,0.0,0.000000,0.0,0.0,0.024390,0.0,0.012195,0.0,0.0,...,0.012195,0.0,0.000000,0.012195,0.0,"Richmond, MA",42.38,-73.37,6,404800


<h3>3. Cluster the potential re-location areas</h3>

In [91]:
# drop zipcode, post_office_city, lat, lng to use only numerical features for clustering
dropcolumns=['zipcode', 'post_office_city', 'lat', 'lng']
df_clustering = dfCluster.drop(dropcolumns, 1)



In [92]:
df_clustering.head()

,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Aquarium,Arcade,...,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,e_school,median_home_value
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.066667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.033333,0.0,6,264800
1,0.0,0.000000,0.0,0.0,0.041667,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.041667,0.0,6,341700
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.095238,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,7,204300
3,0.0,0.036145,0.0,0.0,0.060241,0.0,0.012048,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.012048,0.000000,0.0,6,251300
4,0.0,0.000000,0.0,0.0,0.024390,0.0,0.012195,0.0,0.0,0.0,...,0.0,0.0,0.0,0.012195,0.0,0.000000,0.012195,0.0,6,404800


In [93]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 3, 1, 4, 1, 1, 1])

In [94]:
# add clustering labels back to the original dataframe
dfCluster.insert(0, 'Cluster Labels', kmeans.labels_)


In [95]:
dfCluster.head()

,Cluster Labels,zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,...,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,post_office_city,lat,lng,e_school,median_home_value
0,1,01010,0.0,0.000000,0.0,0.0,0.000000,0.0,0.066667,0.0,...,0.000000,0.0,0.000000,0.033333,0.0,"Brimfield, MA",42.15,-72.20,6,264800
1,4,01054,0.0,0.000000,0.0,0.0,0.041667,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.041667,0.0,"Leverett, MA",42.47,-72.49,6,341700
2,1,01081,0.0,0.000000,0.0,0.0,0.000000,0.0,0.095238,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,"Wales, MA",42.06,-72.23,7,204300
3,1,01238,0.0,0.036145,0.0,0.0,0.060241,0.0,0.012048,0.0,...,0.000000,0.0,0.012048,0.000000,0.0,"Lee, MA",42.30,-73.22,6,251300
4,3,01254,0.0,0.000000,0.0,0.0,0.024390,0.0,0.012195,0.0,...,0.012195,0.0,0.000000,0.012195,0.0,"Richmond, MA",42.38,-73.37,6,404800


Visualize clusters

In [96]:
import numpy as np # library to handle data in a vectorized manner

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[42.292, -71.5], zoom_start=3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfCluster['lat'], dfCluster['lng'], dfCluster['zipcode'], dfCluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The desired locations are in cluster #2.

In [97]:
# create new dataframe to keep only locations that belong to cluster 1 
df_cluster1=dfCluster[dfCluster['Cluster Labels']== 2]
df_cluster1

,Cluster Labels,zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,...,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,post_office_city,lat,lng,e_school,median_home_value
12,2,01451,0.0,0.0,0.00,0.00,0.013889,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.0,"Harvard, MA",42.500,-71.580,6,590900
30,2,01742,0.0,0.0,0.00,0.01,0.060000,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.0,"Concord, MA",42.460,-71.360,7,686700
38,2,01776,0.0,0.0,0.00,0.00,0.050000,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.00,0.0,"Sudbury, MA",42.390,-71.420,6,632800
39,2,01778,0.0,0.0,0.00,0.00,0.040000,0.00,0.0,0.0,...,0.010000,0.000000,0.00,0.00,0.0,"Wayland, MA",42.360,-71.360,6,589700
45,2,01890,0.0,0.0,0.00,0.00,0.010000,0.00,0.0,0.0,...,0.030000,0.000000,0.00,0.00,0.0,"Winchester, MA",42.450,-71.150,6,676800
54,2,02043,0.0,0.0,0.00,0.00,0.070000,0.00,0.0,0.0,...,0.010000,0.000000,0.00,0.00,0.0,"Hingham, MA",42.220,-70.880,6,646700
57,2,02052,0.0,0.0,0.00,0.00,0.032258,0.00,0.0,0.0,...,0.010753,0.010753,0.00,0.00,0.0,"Medfield, MA",42.180,-71.310,6,582800
74,2,02420,0.0,0.0,0.00,0.00,0.080000,0.00,0.0,0.0,...,0.010000,0.000000,0.01,0.00,0.0,"Lexington, MA",42.460,-71.220,6,688500
75,2,02421,0.0,0.0,0.00,0.00,0.080000,0.00,0.0,0.0,...,0.020000,0.000000,0.00,0.00,0.0,"Lexington, MA",42.440,-71.230,6,718300
76,2,02461,0.0,0.0,0.00,0.00,0.060000,0.00,0.0,0.0,...,0.010000,0.000000,0.00,0.02,0.0,"Newton Highlands, MA",42.310,-71.210,6,682600


<h3>4. Conclusion and Recommendations</h3>

Based on the results above, the desired relocation places reside in the cluster 2 - same cluster as current location. Let's Map cluter #2 locations.

In [98]:
# create map
map_relocate = folium.Map(location=[42.292, -71.5], zoom_start=10)

# add markers to the map
for lat, lon, poi, cluster in zip(df_cluster1['lat'], df_cluster1['lng'], df_cluster1['post_office_city'], df_cluster1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_relocate)
       
map_relocate

There are 17 recommended relocation areas that are comparable to the current location.

In [99]:
df_cluster1[['post_office_city','median_home_value','e_school']]

,post_office_city,median_home_value,e_school
12,"Harvard, MA",590900,6
30,"Concord, MA",686700,7
38,"Sudbury, MA",632800,6
39,"Wayland, MA",589700,6
45,"Winchester, MA",676800,6
54,"Hingham, MA",646700,6
57,"Medfield, MA",582800,6
74,"Lexington, MA",688500,6
75,"Lexington, MA",718300,6
76,"Newton Highlands, MA",682600,6


Out of 17 recommended locations there are two with higher school rating and lower home price: Concord, Needham and Vineyard Haven.

Let's find the top 10 venues for those locations.

In [100]:
# define function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
df_cluster2=df_cluster1
df_cluster2.head()

,Cluster Labels,zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,...,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,post_office_city,lat,lng,e_school,median_home_value
12,2,01451,0.0,0.0,0.0,0.00,0.013889,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,"Harvard, MA",42.50,-71.58,6,590900
30,2,01742,0.0,0.0,0.0,0.01,0.060000,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,"Concord, MA",42.46,-71.36,7,686700
38,2,01776,0.0,0.0,0.0,0.00,0.050000,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,"Sudbury, MA",42.39,-71.42,6,632800
39,2,01778,0.0,0.0,0.0,0.00,0.040000,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.0,"Wayland, MA",42.36,-71.36,6,589700
45,2,01890,0.0,0.0,0.0,0.00,0.010000,0.0,0.0,0.0,...,0.03,0.0,0.0,0.0,0.0,"Winchester, MA",42.45,-71.15,6,676800


In [104]:
del df_cluster2['post_office_city']
del df_cluster2['lat']
del df_cluster2['lng']
del df_cluster2['e_school']
del df_cluster2['median_home_value']
df_cluster2.head()

,Cluster Labels,zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
12,2,01451,0.0,0.0,0.0,0.00,0.013889,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
30,2,01742,0.0,0.0,0.0,0.01,0.060000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
38,2,01776,0.0,0.0,0.0,0.00,0.050000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
39,2,01778,0.0,0.0,0.0,0.00,0.040000,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.00,0.01,0.0,0.0,0.0,0.0
45,2,01890,0.0,0.0,0.0,0.00,0.010000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0


In [105]:
del df_cluster2['Cluster Labels']
df_cluster2.head()

,zipcode,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Aquarium,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
12,01451,0.0,0.0,0.0,0.00,0.013889,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
30,01742,0.0,0.0,0.0,0.01,0.060000,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
38,01776,0.0,0.0,0.0,0.00,0.050000,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0
39,01778,0.0,0.0,0.0,0.00,0.040000,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.00,0.01,0.0,0.0,0.0,0.0
45,01890,0.0,0.0,0.0,0.00,0.010000,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0


In [108]:
# create new dataframe with top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['zipcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['zipcode'] = df_cluster2['zipcode']

for ind in np.arange(df_cluster2.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_cluster2.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,01451,Donut Shop,Pizza Place,Sandwich Place,Farm,Italian Restaurant,Intersection,Hotel,Café,Golf Course,Fast Food Restaurant
30,01742,American Restaurant,Historic Site,Pharmacy,Pizza Place,History Museum,Ice Cream Shop,Park,Café,Coffee Shop,Liquor Store
38,01776,Sandwich Place,American Restaurant,Pizza Place,Gym / Fitness Center,Gym,Golf Course,Coffee Shop,Farm,Donut Shop,Bakery
39,01778,Coffee Shop,Grocery Store,American Restaurant,Pizza Place,Italian Restaurant,Bakery,Golf Course,Department Store,Furniture / Home Store,Japanese Restaurant
45,01890,Italian Restaurant,Trail,Ice Cream Shop,Mexican Restaurant,Grocery Store,Park,Deli / Bodega,Bakery,Vegetarian / Vegan Restaurant,Supermarket
54,02043,American Restaurant,Italian Restaurant,Seafood Restaurant,Grocery Store,Pizza Place,Golf Course,Café,Beach,Department Store,Breakfast Spot
57,02052,Donut Shop,Pizza Place,Grocery Store,Bar,Pharmacy,Convenience Store,Coffee Shop,American Restaurant,Restaurant,Sandwich Place
74,02420,American Restaurant,Pizza Place,Italian Restaurant,Grocery Store,Hotel,Coffee Shop,Sandwich Place,Spa,Gym,Supermarket
75,02421,American Restaurant,Italian Restaurant,Grocery Store,Seafood Restaurant,Hotel,Pizza Place,Sandwich Place,Park,Gym,Historic Site
76,02461,Ice Cream Shop,Pizza Place,American Restaurant,Park,Italian Restaurant,Grocery Store,Burger Joint,Liquor Store,Coffee Shop,Japanese Restaurant
